# MCP

In [7]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace, WebSearchTool
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Let's set up our RAG database connection:

In [8]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [9]:
# This is the same code as in the rag.ipynb notebook


@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

Integrate EXA Search as an MCP:

In [4]:
# Exa Search MCP code comes here:
exa_search_mcp = MCPServerStreamableHttp(
     name = "exa search mcp",
     params={
         "url": f"https://mcp.exa.ai/mcp?exaApiKey={os.environ.get('EXA_API_KEY')}",
         "timeout": 30
     },
     client_session_timeout_seconds=30,
     cache_tools_list=True,
     max_retry_attempts=1
)

await exa_search_mcp.connect()

calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool],
    mcp_servers=[exa_search_mcp]
)

Reference query - shouldn't use ExaSearch:

In [6]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in an english breakfast? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    A typical English breakfast (standard version) is about 900 kcal per plate.
    
    Example common ingredients and approximate calories for one serving:
    - 2 eggs: ~140 kcal
    - 2 bacon rashers: ~90 kcal
    - 1 pork sausage: ~230 kcal
    - 1/2 cup baked beans: ~120–150 kcal
    - 1 grilled tomato: ~20 kcal
    - 1/2 cup mushrooms: ~8 kcal
    - 2 slices toast with a little butter: ~150–180 kcal
    - optional butter for frying/on toast: ~30 kcal
    
    Total: roughly 900 kcal (range ~750–1000 kcal depending on portion sizes and choices like white vs. whole-grain toast, leaner sausages, extra toppings, etc.).
    
    Calories per 100 g vary with ingredients and portions but typically around 170–210 kcal per 100 g for a plate that weighs roughly 450–550 g.
    
    If you tell me the exact ingredients and portions you’re using, I’ll calculate a precise total and per-100 g value.
- 2 new i

In [22]:
calorie_agent_with_openai_websearch = Agent(
    name="Nutrition Assistant for meals with openai websearch",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        0) First, use WebSearchTool to figure out the exact ingredients of the meal.
        1) Then, if not available using the WebSearchTool, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use the WebSearchTool to find the exact recipe and ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[WebSearchTool(), calorie_lookup_tool]
)

In [23]:
with trace("Nutrition Assistant with openai websearchtool"):
    result = await Runner.run(
        calorie_agent_with_openai_websearch,
        "How many calories are in total in an english breakfast? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant for meals with openai websearch", ...)
- Final output (str):
    Here’s a representative Full English Breakfast (1 serving) with ingredient quantities and calories:
    
    - Fried Egg: 60 g — 90 kcal
    - Pork Sausages: 110 g — 330 kcal
    - Bacon rashers: 70 g — 250 kcal
    - Baked Beans in tomato sauce: 150 g — 120 kcal
    - Hash browns: 100 g — 280 kcal
    - Fried bread: 80 g — 280 kcal
    - Grilled Tomato: 80 g — 15 kcal
    - Black Pudding: 60 g — 180 kcal
    - Butter: 10 g — 75 kcal
    
    Total: 1,620 kcal for the serving. Calories per 100 g: about 225 kcal. (Exact ingredients and calories taken from a sample Full English Breakfast recipe with per-ingredient breakdown.) ([arise-app.com](https://www.arise-app.com/dish/Full-English-Breakfast?utm_source=openai))
    
    Note: actual calories vary by ingredients and portion sizes.
- 4 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrai